<a href="https://colab.research.google.com/github/tanaymukherjee/Shapley-Value/blob/main/Attribution_Modeling_using_Shapley_value.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Conversion ratio example
Let’s walk through an example. Say that your company converted 100 opportunities at the end of a fiscal quarter. During that period, the marketing department advertised to the associated accounts using three channels:

N = {Facebook, Google, LinkedIn}

All 100 accounts were touched by one or more of the channels throughout their buyer journeys. In other words, the channels worked together by forming coalitions to increase the likelihood of opportunity conversion.

The table below lists all possible channel coalitions and their conversion ratios:

In [1]:
import numpy as np
import pandas as pd
from itertools import combinations

def subsets(S):
    '''Returns all possible subsets of the given set'''
    s = []
    for i in range(1, len(S)+1):
        s.extend(map(list, combinations(S, i)))
    return list(map('+'.join, s))

N = sorted({'Facebook', 'Google', 'LinkedIn'})

coalitions = subsets(N)
coalitions_lbl = ['S{}'.format(i) for i in range(1, len(coalitions)+1)]

# The coalition 'Facebook+Google' (S4) resulted in 10 conversions from 100 opportunities,
# so has a conversion ratio of 10/100 = 0.1 (10%)
IR = np.array([0.18, 0.04, 0.08, 0.1, 0.26, 0.07, 0.27])

pd.options.display.float_format = '{:,.2f}'.format
pd.DataFrame({
    'Coalition': coalitions,
    'Ratio': IR
}, dtype=np.float64, index=coalitions_lbl)

,Coalition,Ratio
S1,Facebook,0.18
S2,Google,0.04
S3,LinkedIn,0.08
S4,Facebook+Google,0.10
S5,Facebook+LinkedIn,0.26
S6,Google+LinkedIn,0.07
S7,Facebook+Google+LinkedIn,0.27


In [2]:
# B is binary squared matrix that represents coalition membership.
# For example, coalition Facebook+LinkedIn (S5) includes members:
# - Facebook (S1)
# - LinkedIn (S3)
# - Facebook+LinkedIn (S5)
# Resulting in the coefficients: [1,0,1,0,1,0,0]

d = 2**len(N)-1
B = np.matrix(np.zeros((d, d)))

for i in range(0, d):
    A = coalitions[i]
    S = subsets(A.split('+'))
    coef = [1 if c in S else 0 for c in coalitions]
    B[i] = coef

pd.options.display.float_format = '{:,.0f}'.format
pd.DataFrame(data=B, index=coalitions, columns=coalitions)

,Facebook,Google,LinkedIn,Facebook+Google,Facebook+LinkedIn,Google+LinkedIn,Facebook+Google+LinkedIn
Facebook,1,0,0,0,0,0,0
Google,0,1,0,0,0,0,0
LinkedIn,0,0,1,0,0,0,0
Facebook+Google,1,1,0,1,0,0,0
Facebook+LinkedIn,1,0,1,0,1,0,0
Google+LinkedIn,0,1,1,0,0,1,0
Facebook+Google+LinkedIn,1,1,1,1,1,1,1


The worth of each coalition is determined by the characteristic function. In this example, the worth is represented as the sum of conversion ratios of each channel in a coalition:

- Coalition S5 = Facebook+LinkedIn
- v(S5) = Facebook (S1) + LinkedIn (S3) + Facebook+LinkedIn (S5)
- v(S5) = 0.18 + 0.08 + 0.26
- v(S5) = 0.52

The coalition containing all players is known as the grand coalition v(N). The grand coalition's worth should be equal to the total payoff.

In [3]:

# The product of the matrices coalition membership and coalition ratios
# is the coalition worth - the result of the characteristic function 'v(S)'

vS = np.dot(B, IR)
vS = np.squeeze(np.asarray(vS))

vSx = ['v({})'.format(lbl) for lbl in coalitions_lbl]
pd.options.display.float_format = '{:,.2f}'.format
pd.DataFrame({
    'Coalition': coalitions,
    'Worth': vS
}, index=vSx)

,Coalition,Worth
v(S1),Facebook,0.18
v(S2),Google,0.04
v(S3),LinkedIn,0.08
v(S4),Facebook+Google,0.32
v(S5),Facebook+LinkedIn,0.52
v(S6),Google+LinkedIn,0.19
v(S7),Facebook+Google+LinkedIn,1.00


### Calculating the Shapley values
Now that we know the worth of each coalition, the Shapley values can be calculated by taking the average of each channels marginal contribution to the game, accounting for all possible orderings. Specifically, the Shapley value gives us a way to distribute the worth of the grand coalition (total payoff) between the three channels.

Facebook's Shapley value:


| Order v(N)                  | Marginal contribution |
| --------------------------- | --------------------- |
| 1. Facebook+Google+LinkedIn | v(S1) = 0.18          |
| 2. Facebook+LinkedIn+Google | v(S1) = 0.18          |
| 3. Google+Facebook+LinkedIn | v(S4) - v(S2) = 0.28  |
| 4. Google+LinkedIn+Facebook | v(S7) - v(S6) = 0.81  |
| 5. LinkedIn+Facebook+Google | v(S5) - v(S3) = 0.44  |
| 6. LinkedIn+Google+Facebook | v(S7) - v(S6) = 0.81  |
| **Average contribution**    | 0.45                  |


- In orders 1 and 2, Facebook is first to arrive so it receives its full contribution
- In order 3, Facebook arrives after Google so its marginal contribution is the coalition containing both channels v(S4)
minus the coalition without Facebook v(S2)
- In orders 4 and 6, Facebook is last to arrive so its marginal contribution is the coalition containing all channels v(S7)
minus the coalition without Facebook v(S6)
- In order 5, Facebook arrives after LinkedIn so its marginal contribution is the coalition containing both channels v(S5)
minus the coalition without Facebook v(S3)

The Shapley values for all channels:

| Order                       | Facebook              | Google                | LinkedIn              |
| --------------------------- | --------------------- | --------------------- | --------------------- |
| 1. Facebook+Google+LinkedIn | v(S1) = 0.18          | v(S4) - v(S1) = 0.14  | v(S7) - v(S4) = 0.68  |
| 2. Facebook+LinkedIn+Google | v(S1) = 0.18          | v(S7) - v(S5) = 0.48  | v(S5) - v(S1) = 0.34  |
| 3. Google+Facebook+LinkedIn | v(S4) - v(S2) = 0.28  | v(S2) = 0.04          | v(S7) - v(S4) = 0.68  |
| 4. Google+LinkedIn+Facebook | v(S7) - v(S6) = 0.81  | v(S2) = 0.04          | v(S6) - v(S2) = 0.15  |
| 5. LinkedIn+Facebook+Google | v(S5) - v(S3) = 0.44  | v(S7) - v(S5) = 0.48  | v(S3) = 0.08          |
| 6. LinkedIn+Google+Facebook | v(S7) - v(S6) = 0.81  | v(S6) - v(S3) = 0.11  | v(S3) = 0.08          |
| **Average contribution**    | 0.45                  | 0.215                 | 0.335                 |

In [4]:
from collections import defaultdict
from math import factorial

# Calculate the Shapley values - the average value of each channel's marginal contribution
# to the grand coalition, taking into account all possible orderings.

shapley = defaultdict(int)
n = len(N)

for i in N:
    for A in coalitions:
        S = A.split('+')
        if i not in S:
            k = len(S) # Cardinality of set |S|
            Si = S
            Si.append(i)
            Si = '+'.join(sorted(Si))
            # Weight = |S|!(n-|S|-1)!/n!
            weight = (factorial(k) * factorial(n-k-1)) / factorial(n)
            # Marginal contribution = v(S U {i})-v(S)
            contrib = vS[coalitions.index(Si)] - vS[coalitions.index(A)]            
            shapley[i] += weight * contrib
    shapley[i] += vS[coalitions.index(i)]/n

In [5]:
pd.options.display.float_format = '{:,.3f}'.format
pd.DataFrame({
    'Shapley value': list(shapley.values())
}, index=list(shapley.keys()))

,Shapley value
Facebook,0.450
Google,0.215
LinkedIn,0.335
